In [1]:
import sys

sys.path.append('./source/')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate, stats
import matplotlib.cm as cm
from numpy import fft
from scipy import interpolate as intp
import importlib
import healpy

from source import plots
from source import prob_dists as pd
from utils import read_param_file, update_params

In [3]:
param_file = './source/n0.params'
params = read_param_file(param_file)
p = params

exposure = 5 * 14000 / (4 * np.pi)
p = update_params(param_file, p, 'exposure', exposure)

phipp = 7.12073e-30
fwimp = phipp * 8 * np.pi / 1e-28
p = update_params(param_file, p, 'fwimp', fwimp) 

print(exposure, fwimp)

5570.423008216337 1.7896346444957159


In [4]:
nside = p['nside']
npix = healpy.nside2npix(nside)
lon, lat = healpy.pix2ang(nside, range(npix), lonlat=True)
ang_dists = np.rad2deg(np.arccos(np.cos(np.deg2rad(lon)) * np.cos(np.deg2rad(lat))))
good_indices = (abs(lat) >= 40)

In [5]:
## S-WAVE
s_psh, s_pshfunc2d, s_fluxes, _ = pd.psh_s(ang_dists[good_indices], return_all=True)
s_psh = s_psh[::2]
s_fluxes = s_fluxes[::2]

s_counts = np.arange(0, 20)

In [6]:
## SOM
som_psh, som_pshfunc2d, som_fluxes, _= pd.psh_som(ang_dists[good_indices], return_all=True)

som_counts = np.arange(0, 50)

In [7]:
# backgrounds
gal_bg = np.load(p['gal_flux_bg_file'])[good_indices] * p['exposure']
iso_bg = p['iso_flux_bg'] * p['exposure']
bg_count = gal_bg + iso_bg

In [8]:
# pc for generating skymaps
s_pc_of_psi = pd.pc_of_psi(p, s_pshfunc2d, s_fluxes, s_counts)
som_pc_of_psi = pd.pc_of_psi(p, som_pshfunc2d, som_fluxes, som_counts)

# choose between s and som
pc_of_psi = s_pc_of_psi 

background = True 



largest count is 65.0 ...this may take a while


In [9]:
# search parameters
num_search = 50
p['fwimp'] *= 10
fwimp_search = (np.log10(p['fwimp'])-2, np.log10(p['fwimp'])+2, num_search)

exposure_search = np.logspace(0, 1.5, num=5) * p['exposure']

In [ ]:
# run likelihoods
s_S = np.zeros((len(exposure_search), num_search))
som_S = np.zeros((len(exposure_search), num_search))
poisson_S = np.zeros((len(exposure_search), num_search))

for i, exposure in enumerate(exposure_search):
    p['exposure'] = exposure

    gal_bg = np.load(p['gal_flux_bg_file'])[good_indices] * p['exposure']
    iso_bg = p['iso_flux_bg'] * p['exposure']
    bg_count = gal_bg + iso_bg
    
    # make the simulated skymap
    psis = np.linspace(40, 180, num=50)
    pc_psi = np.array([integrate.simps(1/fwimp * s_pshfunc2d(abs(psi), s_fluxes).flatten() * stats.poisson.pmf(s_counts[:, np.newaxis], p['exposure'] * fwimp * s_fluxes), fwimp * s_fluxes, axis=-1) for psi in psis])
    pc_of_psi = intp.interp1d(psis, pc_psi, axis=0)
    # generate sky map
#     pc_of_psi = pd.pc_of_psi(p, s_pshfunc2d, s_fluxes, s_counts, bg_count)
    subcounts, ang_dists, bg_counts = pd.generate_skymap_sample_pc(p, pc_of_psi, return_subcounts=True, save_output=True, with_bg=background)
    print('generated skymap with', p['fwimp'])
    print(subcounts)
#     subcounts, ang_dists, bg_counts = pd.generate_skymap_sample_pc_2(p, s_pshfunc2d, s_fluxes, return_subcounts=True, save_output=True, with_bg=background)
    if np.all(subcounts == 0):
        break
    
    counts = np.arange(0, max(subcounts)+1)
    
    S, fwimps = pd.likelihood(p, s_psh, subcounts.astype(np.int16), s_fluxes, counts, fwimp_limits=fwimp_search, bg_count=bg_count)
    s_S[i] = S
    print('finish swave')
    
    S, fwimps = pd.likelihood(p, som_psh, subcounts.astype(np.int16), som_fluxes, counts, fwimp_limits=fwimp_search, bg_count=bg_count)
    som_S[i] = S
    print('finish som')
    
    S, fwimps = pd.poisson_likelihood(p, s_psh, subcounts.astype(np.int16), s_fluxes, counts, fwimp_limits=fwimp_search, bg_count=bg_count)
    poisson_S[i] = S
    print('finish poi')
    
    print(i+1, '/', len(exposure_search))

saved in ./output/n0_skymap_43725.npy
generated skymap with 17.89634644495716
[15. 12.  7. ... 16. 10. 15.]
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0
zero prob pixels 0

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
# ax.set_ylim(bottom=0, top=)
ax.set_yscale('log')
ax.set_ylabel('Exposure [cm2 s]')
ax.set_xscale('log')
ax.set_xlabel('fwimp')
pc = ax.pcolormesh(fwimps, exposure_search, s_S)
fig.colorbar(pc);

In [ ]:
np.amax(s_S, axis=-1), s_S[:, -1]

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(17, 4))
i = 0
for ax, s, lab in zip(axs, [s_S, som_S, poisson_S], ['s-wave L', 'som L', 'poisson L']):
    for expdat, expo in zip(s, exposure_search):
        print(lab, expdat.min(), '\t', p['fwimp'], '\t', fwimps[np.exp(-0.5*(expdat - expdat.min())).argmax()])
        ax.plot(fwimps, np.exp(-0.5*(expdat - expdat.min())), label=f'{expo:.2}cm2s')
#         ax.plot(fwimps, expdat - expdat.min(), label=f'{expo:.2}cm2s')

    ax.set_xscale('log')
    ax.set_xlabel('fwimp')
    ax.set_ylabel('L')
    ax.set_title(lab)
    ax.legend()
#     ax.set_yscale('log')
#     ax.set_ylim(bottom=0)
print(p['fwimp'])

In [ ]:
np.mean(subcounts), np.std(subcounts), len(subcounts)

In [ ]:
subcounts[:100]

In [ ]:
som_S[1]